simple scraper for https://www.2basketballbundesliga.de/spielplan/ .

Goal is to get the Boxscores of all matches from the 2021/22 Season.

First Step:
Match results containing
* Game day
* Game date
* Game start time
* Home team
* Guest team
* result
* link to boxscore
* __maybe__ game id


Classes:
* "slide-entry-title entry-title" -> Gameday

In [77]:
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import csv

In [2]:
page = requests.get("https://www.2basketballbundesliga.de/spielplan/?")
html = page.content

In [3]:
# parse the html and save it into a BeautifulSoup instance
bs = BeautifulSoup(html, 'html.parser')

In [4]:
print(bs.prettify())

<!DOCTYPE html>
<html class="html_stretched responsive av-preloader-disabled av-default-lightbox html_header_top html_logo_left html_main_nav_header html_menu_right html_slim html_header_sticky html_header_shrinking html_mobile_menu_phone html_header_searchicon html_content_align_center html_header_unstick_top_disabled html_header_stretch_disabled html_av-submenu-hidden html_av-submenu-display-click html_av-overlay-side html_av-overlay-side-classic html_av-submenu-noclone html_entry_id_3051 av-cookies-no-cookie-consent av-no-preview html_text_menu_active" lang="de-DE">
 <head>
  <meta charset="utf-8"/>
  <meta content="index, follow" name="robots"/>
  <!-- mobile setting -->
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- Scripts/CSS and wp_head hook -->
  <title>
   Spielplan ProA – BARMER 2. Basketball Bundesliga
  </title>
  <meta content="max-image-preview:large" name="robots">
   <link href="//www.2basketballbundesliga.de" rel="dns-prefetch"/>
   <li

In [43]:
sections = bs.find_all('section')


In [73]:
for section in sections:
    mds = section.find_all(class_="slide-entry-title entry-title")
    mds_list = [md.get_text() for md in mds]
    trs = section.find_all('tr')
    i=1
    for tr in trs:
        items = tr.find_all('td')
        items_list = [item.get_text() for item in items]
        for item in items:
            for a in item.find_all('a', href=True):
                link = a['href']
                id = link[39:-11]
        if len(items_list) > 0:
            print(id +" " +mds_list[0] +" match "+str(items_list)+" link: "+ link)

108931 PO-1/4 Finale Spiel 1 match ['Datum 14.04.2022', 'Uhrzeit 19:30 Uhr', 'Heim Tigers Tübingen', 'Gast Eisbären Bremerhaven', 'Ergebnis 82:78'] link: http://live.2basketballbundesliga.de/g/108931?s=boxscore
108930 PO-1/4 Finale Spiel 1 match ['Datum 22.04.2022', 'Uhrzeit 19:30 Uhr', 'Heim RÖMERSTROM Gladiators Trier', 'Gast Bayer Giants Leverkusen', 'Ergebnis 92:68'] link: http://live.2basketballbundesliga.de/g/108930?s=boxscore
108933 PO-1/4 Finale Spiel 1 match ['Datum 22.04.2022', 'Uhrzeit 19:30 Uhr', 'Heim Medipolis SC Jena', 'Gast Uni Baskets Paderborn', 'Ergebnis 89:84'] link: http://live.2basketballbundesliga.de/g/108933?s=boxscore
108932 PO-1/4 Finale Spiel 1 match ['Datum 26.04.2022', 'Uhrzeit 19:30 Uhr', 'Heim ROSTOCK SEAWOLVES', 'Gast PS Karlsruhe LIONS', 'Ergebnis 107:103'] link: http://live.2basketballbundesliga.de/g/108932?s=boxscore
108936 PO-1/4 Finale Spiel 2 match ['Datum 17.04.2022', 'Uhrzeit 18:00 Uhr', 'Heim Eisbären Bremerhaven', 'Gast Tigers Tübingen', 'Ergeb

In [137]:
header = ['id','season','matchday','date','time','home','guest','result','link']

links = []
# open the file in the write mode
with open('../data/2021_22_matches.csv', 'w', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)

    # write a header to the csv file
    writer.writerow(header)

    for section in sections:
        mds = section.find_all(class_="slide-entry-title entry-title")
        mds_list = [md.get_text() for md in mds]
        trs = section.find_all('tr')
        for tr in trs:
            items = tr.find_all('td')
            items_list = [item.get_text() for item in items]
            for item in items:
                for a in item.find_all('a', href=True):
                    link = a['href']
                    links.append(link)
                    id = link[39:-11]
            if len(items_list) > 0:
                data= [id,"2021_22",mds_list[0], items_list[0][6:], items_list[1][8:-4],items_list[2][5:], items_list[3][5:], items_list[4][9:], link ]
                writer.writerow(data)

    print("Success")



Success


Great, part 1 done

### __Scraping Boxscores__

In [139]:
page = requests.get("https://live.2basketballbundesliga.de/g/108556?s=boxscore")
html = page.content
bs = BeautifulSoup(html, 'html.parser')
bs.prettify()
tables = bs.find_all('tbody')

tables

[<tbody>
 <tr style="text-align: center">
 <td id="score-Q1">-- : --</td>
 <td id="score-Q2">-- : --</td>
 <td id="score-Q3">-- : --</td>
 <td id="score-Q4">-- : --</td>
 </tr>
 </tbody>,
 <tbody>
 <tr id="team-1-stats">
 <td class="teamname"></td>
 <td class="td-2p" style="text-align: center">0</td>
 <td class="td-3p" style="text-align: center">0</td>
 <td class="td-fg" style="text-align: center">0</td>
 <td class="td-ft" style="text-align: center">0</td>
 <td class="td-pts" style="text-align: center">0</td>
 <td class="td-ass" style="text-align: center">0</td>
 <td class="td-reb" style="text-align: center">0</td>
 <td class="td-fls" style="text-align: center">0</td>
 <td class="td-bl" style="text-align: center">0</td>
 <td class="td-st" style="text-align: center">0</td>
 <td class="td-to" style="text-align: center">0</td>
 <td class="td-eff" style="text-align: center">0</td>
 </tr>
 <tr id="team-2-stats">
 <td class="teamname"></td>
 <td class="td-2p" style="text-align: center">0</td

Dynamic website -> Selenium + Chromediver needed
(Terminal, activate environment, then pip install selenium)
https://chromedriver.storage.googleapis.com/index.html?path=103.0.5060.53/
https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de

In [104]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [105]:
website = "https://live.2basketballbundesliga.de/g/108961?s=boxscore"
path = "../drivers/chromedriver"


In [171]:
header = ['matchId','team','number','player','min','2P','2P_A','2P_R','3P','3P_A', '3P_R', 'FG', 'FG_A', 'FG_R', 'FT', 'FT_A', 'FT_R', 'P' , 'AS', 'R_O', 'R_D', 'R_T', 'ST', 'TO','EF' ]
alpha_hdr = ['matchId','team', 'name', 'min', '2Ps','3Ps','FGs','FTs','Ps','As','Rs','Fs','BLs','STs','TOs','EFs']
boxscore = pd.DataFrame(columns=header)
boxscore

,matchId,team,number,player,min,2P,2P_A,2P_R,3P,3P_A,...,FT_A,FT_R,P,AS,R_O,R_D,R_T,ST,TO,EF


In [291]:
alpha_box = pd.DataFrame(columns=alpha_hdr)
alpha_box

,matchId,team,name,min,2Ps,3Ps,FGs,FTs,Ps,As,Rs,Fs,BLs,STs,TOs,EFs


In [254]:
#function to append scraped elements to a list
def sel_to_lst (listname, sels):
    for sel in sels:
        listname.append(sel.text)

In [240]:
driver = webdriver.Chrome(path)
names_lst = []
minutes_lst = []
TwoPs_lst = []
ThreePs_lst = []
FGs_lst = []
FTs_lst = []
Ps_lst = []
As_lst = []
Rs_lst = []
Fs_lst = []
BLs_lst = []
STs_lst = []
TOs_lst = []
EFs_lst = []


driver.get(website)
time.sleep(0.5)
    home_name = driver.find_element(By.ID, "team-1-h1").text
    guest_name = driver.find_element(By.ID, "team-2-h1").text
    home = driver.find_element(By.ID, "team-1-table")
    guest = driver.find_element(By.ID, "team-2-table")


    #Player rows
    names = home.find_elements(By.CLASS_NAME,"td-name")
    sel_to_lst(names_lst, names)
    minutes = home.find_elements(By.CLASS_NAME,"td-min")
    sel_to_lst(minutes_lst, minutes)
    TwoPs = home.find_elements(By.CLASS_NAME,"td-2p")
    sel_to_lst(TwoPs_lst, TwoPs)
    ThreePs = home.find_elements(By.CLASS_NAME,"td-3p")
    sel_to_lst(ThreePs_lst,ThreePs)
    FGs = home.find_elements(By.CLASS_NAME,"td-fg")
    sel_to_lst(FGs_lst,FGs)
    FTs = home.find_elements(By.CLASS_NAME,"td-ft")
    sel_to_lst(FTs_lst, FTs)
    Ps = home.find_elements(By.CLASS_NAME,"td-pts")
    sel_to_lst(Ps_lst,Ps)
    As = home.find_elements(By.CLASS_NAME,"td-ass")
    sel_to_lst(As_lst, As)
    Rs = home.find_elements(By.CLASS_NAME,"td-reb")
    sel_to_lst(Rs_lst, Rs)
    Fs = home.find_elements(By.CLASS_NAME,"td-fls")
    sel_to_lst(Fs_lst, Fs)
    BLs = home.find_elements(By.CLASS_NAME,"td-bl")
    sel_to_lst(BLs_lst,BLs)
    STs = home.find_elements(By.CLASS_NAME,"td-st")
    sel_to_lst(STs_lst, STs)
    TOs = home.find_elements(By.CLASS_NAME,"td-to")
    sel_to_lst(TOs_lst, TOs)
    EFs = home.find_elements(By.CLASS_NAME,"td-eff")
    sel_to_lst(EFs_lst, EFs)

    #Teamaktionen
    team_act = home.find_element(By.CLASS_NAME, "tr-team")
    names_lst.append("Team")
    minutes_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-min").text)
    TwoPs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-2p").text)
    ThreePs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-3p").text)
    FGs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-fg").text)
    FTs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-ft").text)
    Ps_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-pts").text)
    As_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-ass").text)
    Rs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-reb").text)
    Fs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-fls").text)
    BLs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-bl").text)
    STs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-st").text)
    TOs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-to").text)
    EFs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-eff").text)

    #Match ID + Teamname
    match_id = website[40:-11]
    length = len(Rs_lst)

    for i in range(length-1):
        to_append = [ match_id , home_name, names_lst[i], minutes_lst[i]
                    , TwoPs_lst[i], ThreePs_lst[i], FGs_lst[i], FTs_lst[i]
                    , Ps_lst[i], As_lst[i], Rs_lst[i+1], Fs_lst[i], BLs_lst[i]
                    , STs_lst[i], TOs_lst[i], EFs_lst[i] ]
        a_series = pd.Series(to_append, index= alpha_box.columns)
        alpha_box = pd.concat([alpha_box, a_series.to_frame().transpose()], axis=0, ignore_index=True)


driver.quit()

print("Done")
alpha_box

/var/folders/q_/hn04084j05g6swn6gqhgv5fc0000gn/T/ipykernel_1489/794878721.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Done


,matchId,team,name,min,2Ps,3Ps,FGs,FTs,Ps,As,Rs,Fs,BLs,STs,TOs,EFs
0,108961,Tigers Tübingen,#6 Daniel Zacek,00:00,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0,0,0 - 0 - 0,0,0,0,0,0
1,108961,Tigers Tübingen,#7 Timo Lanmüller,18:15,1 - 2 - 50%,1 - 7 - 14%,2 - 9 - 22%,1 - 1 - 100%,6,2,1 - 1 - 2,4,1,0,0,4
2,108961,Tigers Tübingen,#10 Gianni Otto,23:02,0 - 1 - 0%,1 - 3 - 33%,1 - 4 - 25%,3 - 5 - 60%,6,3,2 - 1 - 3,4,0,1,4,4
3,108961,Tigers Tübingen,#11 Erol Antonio Ersek,17:59,2 - 3 - 67%,0 - 3 - 0%,2 - 6 - 33%,1 - 2 - 50%,5,1,1 - 2 - 3,4,1,0,1,4
4,108961,Tigers Tübingen,#12 Aatu Albert Kivimäki,01:33,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0,0,0 - 0 - 0,0,0,0,0,0
5,108961,Tigers Tübingen,#17 Mateo Seric,24:54,2 - 7 - 29%,1 - 6 - 17%,3 - 13 - 23%,4 - 6 - 67%,11,0,0 - 3 - 3,3,0,1,2,1
6,108961,Tigers Tübingen,#26 Till-Joscha Jönke,13:38,2 - 2 - 100%,0 - 1 - 0%,2 - 3 - 67%,1 - 1 - 100%,5,3,1 - 4 - 5,4,0,1,2,11
7,108961,Tigers Tübingen,#31 Daniel Keppeler,25:07,2 - 3 - 67%,1 - 1 - 100%,3 - 4 - 75%,5 - 6 - 83%,12,5,4 - 5 - 9,3,0,2,2,24
8,108961,Tigers Tübingen,#33 Ryan Richard Mikesell,38:15,7 - 9 - 78%,2 - 8 - 25%,9 - 17 - 53%,6 - 6 - 100%,26,2,0 - 5 - 5,2,0,1,3,23
9,108961,Tigers Tübingen,#44 Bakary Dibba,07:52,2 - 3 - 67%,0 - 0 - 0%,2 - 3 - 67%,0 - 0 - 0%,4,0,0 - 0 - 0,1,0,0,2,1


In [281]:
def get_teambox(driver, df, team_name, num, match_id):

    # setup lists
    names_lst = []
    minutes_lst = []
    TwoPs_lst = []
    ThreePs_lst = []
    FGs_lst = []
    FTs_lst = []
    Ps_lst = []
    As_lst = []
    Rs_lst = []
    Fs_lst = []
    BLs_lst = []
    STs_lst = []
    TOs_lst = []
    EFs_lst = []

    #Get table    
    home = driver.find_element(By.ID, "team-"+str(num)+"-table")

    #Player rows
    names = home.find_elements(By.CLASS_NAME,"td-name")
    sel_to_lst(names_lst, names)
    minutes = home.find_elements(By.CLASS_NAME,"td-min")
    sel_to_lst(minutes_lst, minutes)
    TwoPs = home.find_elements(By.CLASS_NAME,"td-2p")
    sel_to_lst(TwoPs_lst, TwoPs)
    ThreePs = home.find_elements(By.CLASS_NAME,"td-3p")
    sel_to_lst(ThreePs_lst,ThreePs)
    FGs = home.find_elements(By.CLASS_NAME,"td-fg")
    sel_to_lst(FGs_lst,FGs)
    FTs = home.find_elements(By.CLASS_NAME,"td-ft")
    sel_to_lst(FTs_lst, FTs)
    Ps = home.find_elements(By.CLASS_NAME,"td-pts")
    sel_to_lst(Ps_lst,Ps)
    As = home.find_elements(By.CLASS_NAME,"td-ass")
    sel_to_lst(As_lst, As)
    Rs = home.find_elements(By.CLASS_NAME,"td-reb")
    sel_to_lst(Rs_lst, Rs)
    Fs = home.find_elements(By.CLASS_NAME,"td-fls")
    sel_to_lst(Fs_lst, Fs)
    BLs = home.find_elements(By.CLASS_NAME,"td-bl")
    sel_to_lst(BLs_lst,BLs)
    STs = home.find_elements(By.CLASS_NAME,"td-st")
    sel_to_lst(STs_lst, STs)
    TOs = home.find_elements(By.CLASS_NAME,"td-to")
    sel_to_lst(TOs_lst, TOs)
    EFs = home.find_elements(By.CLASS_NAME,"td-eff")
    sel_to_lst(EFs_lst, EFs)

    #Teamaktionen
    team_act = home.find_element(By.CLASS_NAME, "tr-team")
    names_lst.append("Team")
    minutes_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-min").text)
    TwoPs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-2p").text)
    ThreePs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-3p").text)
    FGs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-fg").text)
    FTs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-ft").text)
    Ps_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-pts").text)
    As_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-ass").text)
    Rs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-reb").text)
    Fs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-fls").text)
    BLs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-bl").text)
    STs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-st").text)
    TOs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-to").text)
    EFs_lst.append(team_act.find_element(By.CLASS_NAME, "td-team-eff").text)

    #Adding to Dataframe
    length = len(Rs_lst)

    for i in range(length-1):
        to_append = [ match_id , team_name, names_lst[i], minutes_lst[i]
                , TwoPs_lst[i], ThreePs_lst[i], FGs_lst[i], FTs_lst[i]
                , Ps_lst[i], As_lst[i], Rs_lst[i+1], Fs_lst[i], BLs_lst[i]
                , STs_lst[i], TOs_lst[i], EFs_lst[i] ]
        a_series = pd.Series(to_append, index= df.columns)
        df = pd.concat([df, a_series.to_frame().transpose()], axis=0, ignore_index=True)
       
    return df

In [282]:
driver = webdriver.Chrome(path)

driver.get(website)
time.sleep(0.5)
home_name = driver.find_element(By.ID, "team-1-h1").text
guest_name = driver.find_element(By.ID, "team-2-h1").text
match_id = website[40:-11]
#alpha_box = pd.concat([alpha_box, get_teambox(driver, alpha_box, home_name, 1, match_id), get_teambox(driver, alpha_box, guest_name, 2, match_id)], axis=0, ignore_index=True) 
alpha_box = pd.concat([get_teambox(driver, alpha_box, home_name, 1, match_id), get_teambox(driver, alpha_box, guest_name, 2, match_id)], axis=0, ignore_index=True) 
driver.quit()

print("Done")
alpha_box

/var/folders/q_/hn04084j05g6swn6gqhgv5fc0000gn/T/ipykernel_1489/2671269826.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Done


,matchId,team,name,min,2Ps,3Ps,FGs,FTs,Ps,As,Rs,Fs,BLs,STs,TOs,EFs
0,108961,Tigers Tübingen,#6 Daniel Zacek,00:00,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0,0,0 - 0 - 0,0,0,0,0,0
1,108961,Tigers Tübingen,#7 Timo Lanmüller,18:15,1 - 2 - 50%,1 - 7 - 14%,2 - 9 - 22%,1 - 1 - 100%,6,2,1 - 1 - 2,4,1,0,0,4
2,108961,Tigers Tübingen,#10 Gianni Otto,23:02,0 - 1 - 0%,1 - 3 - 33%,1 - 4 - 25%,3 - 5 - 60%,6,3,2 - 1 - 3,4,0,1,4,4
3,108961,Tigers Tübingen,#11 Erol Antonio Ersek,17:59,2 - 3 - 67%,0 - 3 - 0%,2 - 6 - 33%,1 - 2 - 50%,5,1,1 - 2 - 3,4,1,0,1,4
4,108961,Tigers Tübingen,#12 Aatu Albert Kivimäki,01:33,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0,0,0 - 0 - 0,0,0,0,0,0
5,108961,Tigers Tübingen,#17 Mateo Seric,24:54,2 - 7 - 29%,1 - 6 - 17%,3 - 13 - 23%,4 - 6 - 67%,11,0,0 - 3 - 3,3,0,1,2,1
6,108961,Tigers Tübingen,#26 Till-Joscha Jönke,13:38,2 - 2 - 100%,0 - 1 - 0%,2 - 3 - 67%,1 - 1 - 100%,5,3,1 - 4 - 5,4,0,1,2,11
7,108961,Tigers Tübingen,#31 Daniel Keppeler,25:07,2 - 3 - 67%,1 - 1 - 100%,3 - 4 - 75%,5 - 6 - 83%,12,5,4 - 5 - 9,3,0,2,2,24
8,108961,Tigers Tübingen,#33 Ryan Richard Mikesell,38:15,7 - 9 - 78%,2 - 8 - 25%,9 - 17 - 53%,6 - 6 - 100%,26,2,0 - 5 - 5,2,0,1,3,23
9,108961,Tigers Tübingen,#44 Bakary Dibba,07:52,2 - 3 - 67%,0 - 0 - 0%,2 - 3 - 67%,0 - 0 - 0%,4,0,0 - 0 - 0,1,0,0,2,1


In [125]:
driver.quit()

In [153]:
links[:10]

['http://live.2basketballbundesliga.de/g/108931?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108930?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108933?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108932?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108936?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108935?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108934?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108937?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108941?s=boxscore',
 'http://live.2basketballbundesliga.de/g/108939?s=boxscore']

In [ ]:
driver = webdriver.Chrome(path)

driver.get(website)
time.sleep(0.5)
home_name = driver.find_element(By.ID, "team-1-h1").text
guest_name = driver.find_element(By.ID, "team-2-h1").text
match_id = website[40:-11]
#alpha_box = pd.concat([alpha_box, get_teambox(driver, alpha_box, home_name, 1, match_id), get_teambox(driver, alpha_box, guest_name, 2, match_id)], axis=0, ignore_index=True) 
alpha_box = pd.concat([get_teambox(driver, alpha_box, home_name, 1, match_id), get_teambox(driver, alpha_box, guest_name, 2, match_id)], axis=0, ignore_index=True) 
driver.quit()

print("Done")
alpha_box

In [315]:
def get_matchbox(driver, df, csvpath):
    home_name = driver.find_element(By.ID, "team-1-h1").text
    guest_name = driver.find_element(By.ID, "team-2-h1").text
    match_id = link[39:-11]
    new_df = pd.concat([get_teambox(driver, df, home_name, 1, match_id), get_teambox(driver, df, guest_name, 2, match_id)], axis=0, ignore_index=True) 
    #return new_df
    new_df.to_csv(csvpath,sep=";", mode='a', index=False, header=False)
    


In [318]:
import random as rd
csvpath = "../data/boxscore_notformatted.csv"
alpha_box = pd.DataFrame(columns=alpha_hdr)
alpha_box.to_csv(csvpath, sep=";", index=False)

driver = webdriver.Chrome(path)

for link in links:
    driver.get(link)
    time.sleep(1 + rd.randint(0,2))
    get_matchbox(driver, alpha_box, csvpath)

    
driver.quit()

print("Done")

/var/folders/q_/hn04084j05g6swn6gqhgv5fc0000gn/T/ipykernel_1489/160359585.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Done


In [317]:
boxscore = pd.read_csv(csvpath, sep=";")
boxscore.head(50)


,matchId,team,name,min,2Ps,3Ps,FGs,FTs,Ps,As,Rs,Fs,BLs,STs,TOs,EFs
0,108931,Tigers Tübingen,#6 Daniel Zacek,00:00,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0.0,0.0,0 - 0 - 0,0,0.0,0.0,0,0
1,108931,Tigers Tübingen,#7 Timo Lanmüller,19:02,1 - 1 - 100%,2 - 2 - 100%,3 - 3 - 100%,0 - 0 - 0%,8.0,3.0,0 - 4 - 4,3,0.0,2.0,3,14
2,108931,Tigers Tübingen,#10 Gianni Otto,11:46,0 - 1 - 0%,1 - 1 - 100%,1 - 2 - 50%,0 - 0 - 0%,3.0,2.0,0 - 2 - 2,1,0.0,1.0,3,4
3,108931,Tigers Tübingen,#11 Erol Antonio Ersek,17:53,1 - 2 - 50%,2 - 5 - 40%,3 - 7 - 43%,0 - 0 - 0%,8.0,3.0,0 - 1 - 1,1,0.0,1.0,2,7
4,108931,Tigers Tübingen,#12 Aatu Albert Kivimäki,33:16,0 - 2 - 0%,3 - 4 - 75%,3 - 6 - 50%,2 - 4 - 50%,11.0,2.0,0 - 2 - 2,4,0.0,0.0,4,6
5,108931,Tigers Tübingen,#17 Mateo Seric,38:50,8 - 14 - 57%,2 - 6 - 33%,10 - 20 - 50%,2 - 4 - 50%,24.0,1.0,3 - 9 - 12,3,1.0,3.0,2,27
6,108931,Tigers Tübingen,#26 Till-Joscha Jönke,03:47,0 - 0 - 0%,0 - 1 - 0%,0 - 1 - 0%,2 - 4 - 50%,2.0,1.0,0 - 0 - 0,0,0.0,0.0,0,0
7,108931,Tigers Tübingen,#31 Daniel Keppeler,19:19,1 - 2 - 50%,0 - 0 - 0%,1 - 2 - 50%,1 - 2 - 50%,3.0,2.0,1 - 3 - 4,3,1.0,0.0,1,7
8,108931,Tigers Tübingen,#33 Ryan Richard Mikesell,40:02,2 - 6 - 33%,2 - 10 - 20%,4 - 16 - 25%,3 - 4 - 75%,13.0,5.0,1 - 5 - 6,1,0.0,1.0,3,9
9,108931,Tigers Tübingen,#44 Bakary Dibba,02:48,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0 - 0 - 0%,0.0,0.0,0 - 0 - 0,1,0.0,0.0,1,-1


In [306]:
driver.quit()

In [319]:
boxscore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   matchId  116 non-null    int64  
 1   team     116 non-null    object 
 2   name     116 non-null    object 
 3   min      106 non-null    object 
 4   2Ps      106 non-null    object 
 5   3Ps      106 non-null    object 
 6   FGs      106 non-null    object 
 7   FTs      106 non-null    object 
 8   Ps       106 non-null    float64
 9   As       106 non-null    float64
 10  Rs       116 non-null    object 
 11  Fs       116 non-null    int64  
 12  BLs      106 non-null    float64
 13  STs      106 non-null    float64
 14  TOs      116 non-null    int64  
 15  EFs      116 non-null    int64  
dtypes: float64(4), int64(4), object(8)
memory usage: 14.6+ KB


# Getting the results and boxscore links of previous seasons

In [329]:
from selenium.webdriver.support.ui import Select
url = "https://www.2basketballbundesliga.de/spielplan/?"
season_lst = ["Saison 2014/2015", "Saison 2015/2016", "Saison 2016/2017", "Saison 2017/2018", "Saison 2018/2019", "Saison 2019/2020", "Saison 2020/2021", "Saison 2021/2022"]
#season_lst = ["Saison 2014/2015"]
code_lst = []
driver = webdriver.Chrome(path)
driver.get(url)


for season in season_lst:   
    dropdown = Select(driver.find_element(By.NAME,"season"))
    dropdown.select_by_visible_text(season)
    elem = driver.find_element(By.TAG_NAME,"html")
    source_code = elem.get_attribute("outerHTML")
    time.sleep(2)
    code_lst.append(source_code)

    
driver.quit()

/var/folders/q_/hn04084j05g6swn6gqhgv5fc0000gn/T/ipykernel_1489/46303653.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [330]:
len(season_lst) == len(code_lst)

True

In [332]:
links_all = []

In [334]:
for i in range(len(season_lst)):
    bs = BeautifulSoup(code_lst[i], 'html.parser')
    sections = bs.find_all('section')
    season_name = season_lst[i].replace(" ","_").replace("/","_")
    header = ['id','season','matchday','date','time','home','guest','result','link']

    # open the file in the write mode
    with open('../data/'+season_name+'.csv', 'w', encoding='UTF8') as f:
        # create the csv writer
        writer = csv.writer(f)

        # write a header to the csv file
        writer.writerow(header)

        for section in sections:
            mds = section.find_all(class_="slide-entry-title entry-title")
            mds_list = [md.get_text() for md in mds]
            trs = section.find_all('tr')
            for tr in trs:
                items = tr.find_all('td')
                items_list = [item.get_text() for item in items]
                for item in items:
                    for a in item.find_all('a', href=True):
                        link = a['href']
                        links_all.append(link)
                        id = link[39:-11]
                if len(items_list) > 0:
                    data= [id,season_name,mds_list[0], items_list[0][6:], items_list[1][8:-4],items_list[2][5:], items_list[3][5:], items_list[4][9:], link ]
                    writer.writerow(data)

        print(season_name+" success")
    
    

Saison_2014_2015 success
Saison_2015_2016 success
Saison_2016_2017 success
Saison_2017_2018 success
Saison_2018_2019 success
Saison_2019_2020 success
Saison_2020_2021 success
Saison_2021_2022 success


In [335]:
len(links_all)

2076

Cancelled games and "free games" are assigned the boxscore link (and therefore id) of the previous game, which took place

In [336]:
#Clean out doubles from links_all
links_unique = list(dict.fromkeys(links_all))
len(links_unique)

2076

### full boxscore scrape

In [337]:

csvpath = "../data/boxscore_full_notformatted.csv"
alpha_box = pd.DataFrame(columns=alpha_hdr)
alpha_box.to_csv(csvpath, sep=";", index=False)

driver = webdriver.Chrome(path)

for link in links_unique:
    driver.get(link)
    time.sleep(1 + rd.randint(0,2))
    get_matchbox(driver, alpha_box, csvpath)

    
driver.quit()

print("Done")

/var/folders/q_/hn04084j05g6swn6gqhgv5fc0000gn/T/ipykernel_1489/2835701049.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


Done


In [325]:
driver.quit()